In [1]:
import time
import logging
import numpy as np


from opponents import Opponent

In [2]:
logging.basicConfig(level='WARN')

## Rule
* Empty:0, Player(you):1, Opponent:2
* board:
| | | |
| - | - | - |
| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |

In [3]:
def check_win(board, size=3):
    for i in range(size):
        # row
        if board[i*3] != 0 and \
            board[i*3] == board[i*3+1] == board[i*3+2]:
            return board[i*3]
        # column
        if board[i] != 0 and \
            board[i] == board[i+3*1] == board[i+3*2]:
            return board[i]
    # diagonal
    if board[4] != 0:
        if board[0] == board[4] == board[8] or \
            board[2] == board[4] == board[6]:
            
            return board[4]

    return 0



class Board():
    def __init__(self, player, opponent):
        self.board = None
        self.player = player
        self.opponent = opponent # AI from TA
        self.n_player_lose = 0
        self.n_player_win = 0

    def show_available_move(self):
        return np.where(self.board == 0)[0]
    
    def show_board(self):
        print(self.board[:3])
        print(self.board[3:6])
        print(self.board[6:9])
    
    def play(self, player_first=False):
        self.board = np.zeros((9,))
        players = [self.player, self.opponent]
        offset = 0
        if player_first:
            idx = self.player.first_move()
            self.board[idx] = 1
        else:
            idx = self.opponent.first_move()
            self.board[idx] = 2
            offset = 1
        
        for i in range(6):
            current_player = players[(i+offset+1)%2]
            idx = current_player.move(self.board)
            assert self.board[idx] == 0, \
                f"[player {(i+offset+1)%2}] There has been already set"
            self.board[idx] = (i+offset+1)%2 + 1
            
            if i >= 3 and self.is_game_finished():
                break
                
    def is_game_finished(self):
        result = check_win(self.board)
        if result == 0:
            return False
        
        if result == 1:
            self.n_player_win += 1
        
        if result == 2:
            self.n_player_lose += 1
        
        return True

# Testing procedure

In [4]:
TIMEOUT = 10
NUM_RUNS = 100

In [5]:
def profile(func):
    def wrap(*args, **kwargs):
        s = time.time()
        result = func(*args, **kwargs)
        duration = time.time() - s
        if duration > TIMEOUT:
            logging.warning(f"Time Limit Exceeded: {duration}")
        logging.info(f"using {duration} sec")
        return result

    return wrap

In [6]:
from random import choice
class Player():
    def __init__(self):
        pass
    
    def first_move(self):
        return choice(range(9))
    
    def _undo(self, board, last_idx):
        board[last_idx] = 0
    
    def minmax(self, board, act):
        available_moves = np.where(board == 0)[0]
        result = check_win(board)
        if result == 2: # opponent win
            return [-1, None]
        elif result == 1: # player win 
            return [1, None]
        elif result == 0 and len(available_moves) == 0: # no one win and no available moves
            return [0, None]
        
        if act == 'max':
            best_choise = []
            for possible_idx in available_moves:
                possible_board = board
                possible_board[possible_idx] = 1
                out, _ = self.minmax(board, 'min')
                best_choise.append(out)
                self._undo(board, possible_idx)
            return [max(best_choise), best_choise]
        elif act == 'min':
            best_choise = []
            for possible_idx in available_moves:
                possible_board = board
                possible_board[possible_idx] = 2
                out, _ = self.minmax(board, 'max')
                best_choise.append(out)
                self._undo(board, possible_idx)
            return [min(best_choise), best_choise]
    
    @profile
    def move(self, board):
        # time.sleep(10)
        available_moves = np.where(board == 0)[0]
        best, choise = self.minmax(board, 'max')
        best_choise = np.where(np.array(choise) == best)[0][0]
        ch = available_moves[best_choise]
        return ch

In [7]:
opponent = Opponent('random')
player = Player()
b = Board(player, opponent)

for i in range(NUM_RUNS):
    player_first = i%2
    b.play(player_first=player_first)
    
    

print(f"you lose {b.n_player_lose} out of {NUM_RUNS} games")

you lose 0 out of 100 games
